In [1]:
import numpy as np
import pandas as pd
import tqdm
from scipy import signal
from pathlib import Path

from dataset_processor import (
    AddGravityColumn,
    Convert_G_to_Ms2,
    ButterworthFilter,
    Resampler,
    Windowize,
    AddStandardActivityCode,
    RenameColumns,
    SplitGuaranteeingAllClassesPerSplit,
    BalanceToMinimumClass,
    Pipeline
)

In [2]:
def read_motionsense(motionsense_path: str) -> pd.DataFrame:
    """Le o dataset do motionsense e retorna um dataframe com os dados (vindos de todos os arquivos CSV)
    O dataframe retornado possui as seguintes colunas:
    - attitude.roll: Rotação em torno do eixo x
    - attitude.pitch: Rotação em torno do eixo y
    - attitude.yaw: Rotação em torno do eixo z
    - gravity.x: Gravidade em torno do eixo x
    - gravity.y: Gravidade em torno do eixo y
    - gravity.z: Gravidade em torno do eixo z
    - rotationRate.x: Velocidade angular em torno do eixo x
    - rotationRate.y: Velocidade angular em torno do eixo y
    - rotationRate.z: Velocidade angular em torno do eixo z
    - userAcceleration.x: Aceleração no eixo x
    - userAcceleration.y: Aceleração no eixo y
    - userAcceleration.z: Aceleração no eixo z
    - activity code: Código da atividade
    - index: Índice da amostra vindo do csv
    - user: Usuário que realizou a atividade
    - serial: Número de série da atividade
    - csv: Caminho do csv que contém a atividade

    Parameters
    ----------
    motionsense_path : str
        Caminho para o dataset MotionSense

    Returns
    -------
    pd.DataFrame
        Dataframe com os dados do dataset MotionSense
    """

    motionsense_path = Path(motionsense_path)
    activity_names = {0: "dws", 1: "ups", 2: "sit", 3: "std", 4: "wlk", 5: "jog"}
    activity_codes = {v: k for k, v in activity_names.items()}

    feature_dtypes = {
        "attitude.roll": np.float32,
        "attitude.pitch": np.float32,
        "attitude.yaw": np.float32,
        "gravity.x": np.float32,
        "gravity.y": np.float32,
        "gravity.z": np.float32,
        "rotationRate.x": np.float32,
        "rotationRate.y": np.float32,
        "rotationRate.z": np.float32,
        "userAcceleration.x": np.float32,
        "userAcceleration.y": np.float32,
        "userAcceleration.z": np.float32,
    }

    dfs = []
    for i, f in enumerate(sorted(motionsense_path.rglob("*.csv"))):
        # Pegando o nome da atividade
        activity_name = f.parents[0].name
        # Pariticiona o nome da atividade em o cóigo da corrida
        activity_name, serial = activity_name.split("_")
        activity_code = activity_codes[activity_name]

        user = int(f.stem.split("_")[1])
        df = pd.read_csv(
            f, names=list(feature_dtypes.keys()), dtype=feature_dtypes, skiprows=1
        )

        # ----- Adiciona colunas auxiliares e meta-dados ------
        df["activity code"] = activity_code
        df["index"] = range(len(df))
        df["user"] = user
        df["serial"] = serial
        df["csv"] = "/".join(f.parts[-2:])
        # ----------------------------------------------------
        dfs.append(df)

    return pd.concat(dfs)


In [3]:
# Caminho para o dataset MotionSense
motionsense_path = Path("data/raw/MotionSense/A_DeviceMotion_data")
# Caminho para salvar o dataset pré-processado
output_path = Path("data/processed/MotionSense")
# Cria o caminho de saída se ele não existir
output_path.mkdir(parents=True, exist_ok=True)

# Dicionário com o mapeamento das colunas para renomear
columns_to_rename = {
    "userAcceleration.x": "accel-x",
    "userAcceleration.y": "accel-y",
    "userAcceleration.z": "accel-z",
    "rotationRate.x": "gyro-x",
    "rotationRate.y": "gyro-y",
    "rotationRate.z": "gyro-z",
}

# Lista com as colunas que são features
feature_columns = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z",
    "attitude.roll",
    "attitude.pitch",
    "attitude.yaw",
    "gravity.x",
    "gravity.y",
    "gravity.z",
]

# Nome das colunas que serão usada para agrupar as janelas
column_group = "csv"

# activity code: standard activity code
standard_activity_code_map = {
    0: 4,
    1: 3,
    2: 0,
    3: 1,
    4: 2,
    5: 5
}


## Bruto

In [4]:
# Lê o dataset
dataframe = read_motionsense(motionsense_path)

# Instancia o objeto para renomear as colunas
renamer = RenameColumns(
    columns_map=columns_to_rename,    # Dicionário com o mapeamento das colunas
)

# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,     # Nome das colunas que serão usadas como features
    samples_per_window=150,                 # Numero de amostras por janela
    samples_per_overlap=0,                  # Numero de amostras que se sobrepõem
    groupby_column=column_group,            # Agrupa pela coluna do CSV. As janelas são criadas para cada grupo da coluna CSV
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Renomeia as colunas
# 2. Cria as janelas
# 3. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        renamer,
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df = pipeline(dataframe)
# Salva os dados
new_df.to_csv(output_path / "raw_unbalanced.csv", index=False)
new_df

Executing RenameColumns
Executing Windowize


Creating windows: 100%|██████████| 360/360 [00:40<00:00,  8.91it/s]


Executing AddStandardActivityCode


,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gravity.z-146,gravity.z-147,gravity.z-148,gravity.z-149,csv,user,activity code,index,serial,standard activity code
0,0.294894,0.219405,0.010714,-0.008389,0.199441,0.168241,0.079382,0.06936,0.072889,0.098347,...,-0.087946,-0.072701,-0.062489,-0.057112,dws_1/sub_1.csv,1,0,0,1,4
1,0.422041,0.489289,0.405519,0.330101,0.278058,0.26667,-0.038128,-0.113882,0.040205,0.70531,...,-0.429028,-0.438791,-0.427743,-0.398845,dws_1/sub_1.csv,1,0,150,1,4
2,0.050002,0.279396,0.397511,0.489769,0.376962,0.086257,0.297043,0.189549,-0.11724,-0.219285,...,-0.060859,-0.035123,-0.006991,0.021725,dws_1/sub_1.csv,1,0,300,1,4
3,0.453086,0.428134,0.145774,0.035071,-0.073498,-0.076478,-0.071926,-0.070026,-0.09827,-0.088027,...,0.034487,0.025593,0.009984,-0.012116,dws_1/sub_1.csv,1,0,450,1,4
4,-0.12326,-0.197224,0.104588,0.464974,0.476676,-0.173624,-0.311906,-0.396358,-0.354741,-0.343695,...,0.0168,-0.017304,-0.065986,-0.116997,dws_1/sub_1.csv,1,0,600,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9237,-0.219509,0.039405,0.177356,0.163627,0.080003,-0.014922,-0.057913,0.004133,0.091218,0.165302,...,-0.615996,-0.563828,-0.51997,-0.519647,wlk_8/sub_9.csv,9,4,3900,8,2
9238,0.41574,0.001829,-0.467739,-0.385918,0.186728,0.043864,-0.726447,-0.768545,-0.734806,-0.569898,...,-0.329562,-0.392729,-0.444477,-0.492672,wlk_8/sub_9.csv,9,4,4050,8,2
9239,-0.128104,0.136487,0.138946,-0.044175,-0.09986,-0.104913,-0.211261,-0.335558,-0.412587,-0.413217,...,-0.293291,-0.255729,-0.223467,-0.191943,wlk_8/sub_9.csv,9,4,4200,8,2
9240,-0.198929,-0.270276,-0.092972,0.201214,0.290746,0.130444,-0.023268,0.024589,0.179717,0.089722,...,-0.643083,-0.595136,-0.550171,-0.534283,wlk_8/sub_9.csv,9,4,4350,8,2


In [5]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.8,
    random_state=42
)(new_df[new_df["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "raw_balanced" 
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)

## Normatizado com reamostrador Resampler

In [6]:
# Lê o dataset
dataframe = read_motionsense(motionsense_path)

# Instancia o objeto para renomear as colunas
renamer = RenameColumns(
    columns_map=columns_to_rename,    # Dicionário com o mapeamento das colunas
)

# Instancia o objeto que adiciona a gravidade
add_gravity = AddGravityColumn(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro
    gravity_columns=["gravity.x", "gravity.y", "gravity.z"],    # Nome das colunas da gravidade
)

# Instancia o objeto que converte a aceleração para m/s²
conversor = Convert_G_to_Ms2(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro que serão convertidas
)

# Instancia o objeto que aplica o filtro Butterworth
butterworth = ButterworthFilter(
    axis_columns=["accel-x", "accel-y", "accel-z"],             # Nome das colunas do aceletômetro em que o filtro será aplicado
    fs=50                                                       # Frequência de amostragem original
)

# Instacia o objeto que reamostra os dados para 20Hz (supondo que o dataset original é 50Hz, constante)
resampler = Resampler(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    original_fs=50,                                             # Frequência de amostragem original
    target_fs=20,                                               # Frequência de amostragem desejada
    groupby_column=column_group,                                # Agrupa pela coluna do CSV. A reamostragem é feita para cada grupo da coluna CSV
)

# Instancia o objeto que cria as janelas
windowizer = Windowize(
    features_to_select=feature_columns,                         # Nome das colunas que serão usadas como features
    samples_per_window=60,                                      # Numero de amostras por janela 
    samples_per_overlap=0,                                      # Numero de amostras que se sobrepõem
    groupby_column=column_group,                                # Agrupa pela coluna do CSV. As janelas são criadas para cada grupo da coluna CSV
)

# Instancia o objeto que adiciona a coluna com o código da atividade
standard_label_adder = AddStandardActivityCode(standard_activity_code_map)

# Cria o pipeline
# 1. Renomeia as colunas
# 2. Adiciona a coluna com a gravidade
# 3. Converte a aceleração para m/s²
# 4. Aplica o filtro Butterworth
# 5. Reamostra os dados para 20Hz
# 6. Cria as janelas
# 7. Adiciona a coluna com o código da atividade
pipeline = Pipeline(
    [
        renamer,
        add_gravity,
        conversor,
        butterworth,
        resampler,
        windowizer,
        standard_label_adder
    ]
)

# Executa o pipeline
new_df_normalized = pipeline(dataframe)
# Salva os dados
new_df_normalized.to_csv(output_path / "standartized_unbalanced.csv", index=False)
new_df_normalized

Executing RenameColumns
Executing AddGravityColumn
Executing Convert_G_to_Ms2
Executing ButterworthFilter
Executing Resampler


Resampling: 100%|██████████| 360/360 [00:03<00:00, 91.42it/s] 


Executing Windowize


Creating windows: 100%|██████████| 360/360 [00:39<00:00,  9.01it/s]


Executing AddStandardActivityCode


,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,accel-x-9,...,gravity.z-57,gravity.z-58,gravity.z-59,csv,user,level_0,activity code,index,serial,standard activity code
0,2.288706,-2.198357,-0.291764,-1.287203,-1.352125,-2.150854,-0.56075,-5.668985,-4.820846,-4.712508,...,-0.168824,-0.107589,-0.064963,dws_1/sub_1.csv,1,0,0,0,1,4
1,2.36918,2.071391,-0.186089,-1.266858,9.145859,-3.198091,-8.636468,-5.221671,-5.435234,0.85808,...,-0.2317,-0.403346,-0.429845,dws_1/sub_1.csv,1,60,0,60,1,4
2,0.169915,2.574639,1.753801,-1.669721,-1.080261,1.21426,-0.776808,5.146641,2.479351,0.875143,...,-0.093774,-0.076443,-0.012494,dws_1/sub_1.csv,1,120,0,120,1,4
3,3.841507,-0.419048,-1.278237,-1.922591,-0.848369,1.197113,2.96001,0.81453,3.038822,1.065182,...,0.078148,0.03872,0.015126,dws_1/sub_1.csv,1,180,0,180,1,4
4,-0.865185,4.523849,-0.97776,-3.88586,-3.891317,-0.431525,3.136046,-2.38584,-4.333047,2.620828,...,0.033785,0.032218,-0.063874,dws_1/sub_1.csv,1,240,0,240,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9237,1.607619,0.623945,0.465307,4.262094,2.833578,4.56784,-4.614095,-3.875252,2.925356,0.955948,...,-0.618384,-0.519307,-0.56834,wlk_8/sub_9.csv,9,1560,4,1560,8,2
9238,-1.221419,-2.253459,-7.314236,2.845808,1.154034,-2.558872,2.021852,0.877564,2.318579,1.460885,...,-0.329568,-0.470694,-0.577037,wlk_8/sub_9.csv,9,1620,4,1620,8,2
9239,0.999744,-1.289663,-2.466112,-3.615957,2.699464,3.932474,-2.826409,-0.488493,-3.259519,-4.276027,...,-0.28724,-0.201253,-0.136173,wlk_8/sub_9.csv,9,1680,4,1680,8,2
9240,2.468716,1.220354,1.352487,5.619042,3.506585,-10.201177,-0.851379,2.019488,0.121718,-1.311952,...,-0.626457,-0.540223,-0.582819,wlk_8/sub_9.csv,9,1740,4,1740,8,2


In [7]:
# Balanceia os dados (exlcuindo as linhas com atividade -1)
train_df, test_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.8,
    random_state=42
)(new_df_normalized[new_df_normalized["standard activity code"] != -1])

train_df, val_df = SplitGuaranteeingAllClassesPerSplit(
    column_to_split="user", 
    class_column="standard activity code", 
    train_size=0.9,
    random_state=42
)(train_df)

balancer = BalanceToMinimumClass(class_column="standard activity code")
train_df = balancer(train_df)
val_df = balancer(val_df)
test_df = balancer(test_df)

ouptut_dir = output_path / "standartized_balanced" 
ouptut_dir.mkdir(parents=True, exist_ok=True)

train_df.to_csv(ouptut_dir / "train.csv", index=False)
val_df.to_csv(ouptut_dir / "validation.csv", index=False)
test_df.to_csv(ouptut_dir / "test.csv", index=False)